(case_kvlcc2_propeller)=
### Propeller

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (10,10)
from src.symbols import *

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]

from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force, fit_motions, create_model_from_motion_regression, create_full_model_from_motion_regression
from wPCC_pipeline.pipelines.prediction.nodes import simulate_euler
from src.models.vmm import VMM
from src.parameters import df_parameters
from src.models.propeller import fit, predict, predictor
from wPCC_pipeline.pipelines.kvlcc2.nodes import fit_propeller_characteristics
import src.models.MMG_propeller as MMG_propeller
from src.substitute_dynamic_symbols import run
from src.models.propeller import preprocess
import src.models.propeller as propeller
from sklearn.metrics import r2_score

import statsmodels.api as sm
from myst_nb import glue
from IPython.display import display, Math, Latex

In [ ]:
ship="kvlcc2_hsva"
vmm_name = "vmm_martins_simple"
#vmm_name = "vmm_abkowitz"
vmm = catalog.load(vmm_name)
ship_data = catalog.load(f"{ship}.ship_data")
data = catalog.load(f"{ship}.updated.joined.data_ek_smooth")
added_masses = catalog.load(f"{ship}.added_masses")
exclude_parameters = catalog.load(f"params:{ship}.motion_regression.exclude_parameters")
open_water_characteristics = catalog.load(f"{ship}.open_water_characteristics")
propeller_coefficients = catalog.load(f"{ship}.propeller_coefficients")

In [ ]:
glue('eqT', propeller.eq_T)
glue('eqKT', propeller.eq_K_T)
glue('eqJ', propeller.eq_J)
glue('eq_MMG_w_p', MMG_propeller.eq_w_p)
glue('eqbetap', Latex(r'$\beta_p=\beta - \frac{r}{U} \cdot x_p $'))

The propeller model for KVLCC2 is developed based on MMG model {cite:p}`yasukawa_introduction_2015-1` where the thrust is expressed as:

```{glue:math} eqT
:label: "eqT"
```

And thrust coefficient $K_T$ is modelled as a second order polynomial:

```{glue:math} eqKT
:label: "eqKT"
```

The coefficients of this polynomial where regressed from the propeller characteristics KVLCC2 propeller characteristic SIMMAN2008 conference {cite:p}`stern_experience_2011` ($k_0$:{glue:}`k_0`, $k_1$:{glue:}`k_1`, $k_2$:{glue:}`k_2`).

```{glue:figure} fig_propeller_characteristic
:figwidth: 1000px
:name: "fig_propeller_characteristic"

KVLCC2 propeller characteristic SIMMAN2008 conference {cite:p}`stern_experience_2011`
```

The advance ration $J$ is calculated as:
```{glue:math} eqJ
:label: "eqJ"
```
where $w_p$ is the wake fraction which can be calculated with the MMG model as: 

```{glue:math} eq_MMG_w_p
:label: "eq_MMG_w_p"
```
Coefficient $C_2$ can have different values depending on the sign of $\beta_p$ due to propeller asymmetry.
The wake fraction $w_p$ is modelled as function of the drift angle $\beta$, yaw rate $r$ and propeller longitudinal position $x_p$:
```{glue:math} eqbetap
:label: "eqbetap"
```

In [ ]:
for key,value in ship_data.items():
    glue(key,value, display=False)
    print(key)

In [ ]:
for key,value in propeller_coefficients.items():
    glue(key,np.round(value, 5), display=False)
    print(key)

In [ ]:
fig,ax=plt.subplots()
open_water_characteristics.plot(ax=ax)
pred_Kt = np.polyval(np.flipud(pd.Series(propeller_coefficients).values), 
           open_water_characteristics.index)
ax.plot(open_water_characteristics.index, pred_Kt, 'r--', label='polynomial')
ax.legend()
ax.grid(True)
glue("fig_propeller_characteristic",fig, display=False)

In [ ]:
eq = MMG_propeller.X_P_solution[0][thrust]
lambda_thrust = sp.lambdify(list(eq.free_symbols), eq)

In [ ]:
ship_data_ = ship_data.copy()
#ship_data_['w_p0'] = 0.5

In [ ]:
ids = data['id'].unique()

np.random.seed(44)
ids_test = np.random.choice(ids, size=int(np.ceil(len(ids)*0.5)), replace=False)
ids_train = list(set(ids) - set(ids_test))

mask = data['id'].isin(ids_train)
df_train = data.loc[mask].copy()

In [ ]:
def train_MMG(df_train):
    
    df_train_MMG = df_train.copy()
    df_train_MMG = preprocess(df_train_MMG, ship_data=ship_data, propeller_coefficients=propeller_coefficients)
    df_train_MMG['C_2'] = np.where(df_train_MMG['beta_p'] > 0, ship_data_["C_2_beta_p_pos"], ship_data_["C_2_beta_p_neg"])
    
    df_train_MMG['thrust_'] = run(function=lambda_thrust, inputs=df_train_MMG, **ship_data_, **propeller_coefficients)
    X = df_train_MMG[['thrust_']]
    y = df_train['thrust']
    model = sm.OLS(y,X)
    predictor_MMG = model.fit()
    return predictor_MMG

def train_simple(df_train):
    df_train_simple = df_train.copy()
    df_train_simple['thrust_'] = run(propeller.lambda_thrust_simple, inputs=df_train_simple, **ship_data_, **propeller_coefficients, w_p=ship_data['w_p0'])
    X = df_train_simple[['thrust_']]
    y = df_train['thrust']
    model = sm.OLS(y,X)
    predictor_simple = model.fit()
    return predictor_simple

In [ ]:
predictor_MMG = train_MMG(df_train=df_train)
predictor_simple = train_simple(df_train=df_train)

add_constant=False
model_pos, model_neg = propeller.fit(data=df_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant)


In [ ]:
mask = data['id'].isin(ids_test)
df_test = data.loc[mask].copy()

In [ ]:
def predict_MMG(df_test):
    df_predict_MMG = df_test.copy()
    df_predict_MMG = preprocess(df_predict_MMG, ship_data=ship_data, propeller_coefficients=propeller_coefficients)
    df_predict_MMG['C_2'] = np.where(df_predict_MMG['beta_p'] > 0, ship_data_["C_2_beta_p_pos"], ship_data_["C_2_beta_p_neg"])
    df_predict_MMG['thrust_'] = run(function=lambda_thrust, inputs=df_predict_MMG, **ship_data_, **propeller_coefficients)
    X = df_predict_MMG[['thrust_']]
    df_predict_MMG['thrust'] = predictor_MMG.predict(X)
    return df_predict_MMG

In [ ]:
def predict_simple(df_test):
    df_predict_simple = df_test.copy()
    df_predict_simple['thrust_'] = run(propeller.lambda_thrust_simple, inputs=df_predict_simple, **ship_data_, **propeller_coefficients, w_p=ship_data['w_p0'])
    X = df_predict_simple[['thrust_']]
    y = df_test['thrust']
    df_predict_simple['thrust'] = predictor_simple.predict(X)
    return df_predict_simple

In [ ]:
df_predict_MMG = predict_MMG(df_test=df_test)
df_predict_simple = predict_simple(df_test=df_test)
df_predict = propeller.predict(model_pos=model_pos, model_neg=model_neg, data=df_test, propeller_coefficients=propeller_coefficients, ship_data=ship_data)

In [ ]:
C_3,C_4 = sp.symbols("C_3 C_4")
eq_model = sp.Eq(w_p, w_p0 + C_1*delta+C_2*delta**2 + C_3*beta_p**2 + C_4*u)
glue("eqmodel",eq_model)

The thrust signal from the model test has been split into a train and validation set. Three different models were tested in the validation:
* MMG model with:
  * $w_{p0}$ = {glue:}`w_p0`
  * $C_1$={glue:}`C_1`
  * $C_2$={glue:}`C_2_beta_p_pos` when $\beta_p>0$ and $C_2$={glue:}`C_2_beta_p_neg` when $\beta_p<=0$
* Simple model:
  * $w_{p}= w_{p0}$
* Polynomial propeller model:
```{glue:math} eqmodel
:label: "eqmodel"
```

The constant interception term was regressed to the MMG and Simple model, apart from that original valued for $w_{p0}$ etc. was kept.

The Polynomial propeller model was developed with polynomial regression and cross validation to make the best feature selection. 
An example from a part of the validation result is shown in [fig](fig_propeller_validation). It seems that the Polynomial propeller model gives the best representation of the KVLCC2 propeller thrust which is also confirmed from a random cross validation giving the following $r^2$ values {glue:}`r2_simple`, {glue:}`r2_MMG`, {glue:}`r2_model` for the simple, MMG and the Polynomial propeller model.

```{glue:figure} fig_propeller_validation
:name: "fig_propeller_validation"

Validation of various propeller models for KVLCC2`
```

The Polynomial propeller model is selected to be used as input to the KVLCC2 VMM based on this cross validation.

In [ ]:
fig,ax=plt.subplots()
mask = df_test['id'] == df_test['id'].unique()[-1]
df_test.loc[mask].plot(y='thrust', ax=ax, label='True')
df_predict_MMG.loc[mask].plot(y='thrust', ax=ax, label='MMG')
df_predict_simple.loc[mask].plot(y='thrust', ax=ax, label='simple')
df_predict.loc[mask].plot(y='thrust', ax=ax, label='model')
ax.set_xlabel('time [s]')
ax.set_ylabel('thrust [N]');
glue("fig_propeller_validation", fig, display=False)

In [ ]:
model_pos.summary()

In [ ]:
model_neg.summary()

In [ ]:
scores = []
for i in range(100):
    
    np.random.seed(i)
    ids_test = np.random.choice(ids, size=int(np.ceil(len(ids)*0.5)), replace=False)
    ids_train = list(set(ids) - set(ids_test))
    
    mask = data['id'].isin(ids_train)
    df_train = data.loc[mask].copy()
    
    mask = data['id'].isin(ids_test)
    df_test = data.loc[mask].copy()
    
    
    # Train
    predictor_MMG = train_MMG(df_train=df_train)
    predictor_simple = train_simple(df_train=df_train)

    add_constant=False
    model_pos, model_neg = propeller.fit(data=df_train, ship_data=ship_data, propeller_coefficients=propeller_coefficients, add_constant=add_constant)

    # Predict
    df_predict_MMG = predict_MMG(df_test=df_test)
    df_predict_simple = predict_simple(df_test=df_test)
    df_predict = propeller.predict(model_pos=model_pos, model_neg=model_neg, data=df_test, propeller_coefficients=propeller_coefficients, ship_data=ship_data)
   
    score = {}
    score['MMG'] = r2_score(y_true=df_test['thrust'], y_pred=df_predict_MMG['thrust'])
    score['simple'] = r2_score(y_true=df_test['thrust'], y_pred=df_predict_simple['thrust'])
    score['model'] = r2_score(y_true=df_test['thrust'], y_pred=df_predict['thrust'])
    scores.append(score)
    
    
scores= pd.DataFrame(scores)
    

In [ ]:
scores.describe()

In [ ]:
for key,values in scores.items():
    r2 = np.round(values.mean(),2)
    glue(f"r2_{key}", r2)
